# PREPARATION

## Import Required Library

In [1]:
import math
from itertools import permutations
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias['type'] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_alternative):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights['weight'] = get_matrix()

        with output:
            print("Weight Criteria:")
            print(weights)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Starter Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

Define criterias

In [6]:
criterias = pd.DataFrame(None, index=criteria_labels, columns=['type'])
print(criterias)

   type
C1  NaN
C2  NaN
C3  NaN
C4  NaN
C5  NaN


Decide which criteria is benefit or cost

In [7]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Create matrix for decision

In [8]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

print(decision_matrix)

    C1  C2  C3  C4  C5
A1   0   0   0   0   0
A2   0   0   0   0   0
A3   0   0   0   0   0
A4   0   0   0   0   0
A5   0   0   0   0   0


In [9]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Normalization

Formula:

**Benefit criteria:**
$$
r_{ij} = \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

**Cost criteria:**
$$
r_{ij} = 1 - \frac{x_{ij}}{\sqrt{\Sigma_{i=1}^m x^2_{ij}}}
$$

In [10]:
matrix_r = decision_matrix.copy(deep=True).astype(float)

for i in range(total_criteria):
    sum_col = (decision_matrix.iloc[:, i] ** 2).sum()

    for j, row in enumerate(decision_matrix.itertuples(index=False)):
        if criterias.iloc[i]['type'] == 'Benefit':
            matrix_r.iloc[j, i] = decision_matrix.iloc[j, i] / math.sqrt(sum_col) # benefit criteria normalization
        else:
            matrix_r.iloc[j, i] = 1 - (decision_matrix.iloc[j, i] / math.sqrt(sum_col)) # cost criteria normalization

print(matrix_r)

          C1        C2        C3        C4        C5
A1  0.617080  0.336265  0.441129  0.564239  0.378766
A2  0.731956  0.378298  0.330847  0.664799  0.291358
A3  0.655372  0.378298  0.551411  0.530719  0.378766
A4  0.463912  0.588464  0.477890  0.463679  0.466173
A5  0.387328  0.504398  0.404368  0.564239  0.640988


# Define Weights

In [11]:
weights = pd.DataFrame(0, index=criteria_labels, columns=['weight'])
print(weights)

    weight
C1       0
C2       0
C3       0
C4       0
C5       0


In [12]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Weighted Matrix

Formula:

$$
v_{ij} = r_{ij} \cdot w_{ij}
$$

In [13]:
weighted_matrix = matrix_r.mul(weights['weight'], axis=1)
print(weighted_matrix)

           C1        C2        C3         C4         C5
A1   9.256203  3.362651  5.734674  10.156302   6.060252
A2  10.979342  3.782982  4.301006  11.966386   4.661732
A3   9.830583  3.782982  7.168343   9.552941   6.060252
A4   6.958684  5.884639  6.212564   8.346218   7.458772
A5   5.809925  5.043976  5.256785  10.156302  10.255811


# Get The Concordance and Discordance Set

Calculate the permutation of alternative to see how many group should concordance and discordance have

In [14]:
permutation_alternative = math.perm(total_alternative, 2)
print(permutation_alternative)

20


## Concordance

In [15]:
concordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] >= weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            concordances_data.append(i)
    else:
        concordances_data.append([a, b, None])

concordances = pd.DataFrame(concordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(concordances)

   First Second Selected Criteria
0     A1     A2                C3
1     A1     A2                C5
2     A1     A3                C4
3     A1     A3                C5
4     A1     A4                C1
5     A1     A4                C4
6     A1     A5                C1
7     A1     A5                C3
8     A1     A5                C4
9     A2     A1                C1
10    A2     A1                C2
11    A2     A1                C4
12    A2     A3                C1
13    A2     A3                C2
14    A2     A3                C4
15    A2     A4                C1
16    A2     A4                C4
17    A2     A5                C1
18    A2     A5                C4
19    A3     A1                C1
20    A3     A1                C2
21    A3     A1                C3
22    A3     A1                C5
23    A3     A2                C2
24    A3     A2                C3
25    A3     A2                C5
26    A3     A4                C1
27    A3     A4                C3
28    A3     A

## Discordance

In [16]:
discordances_data = []
for a, b in permutations(alternative_labels, 2):
    row_1 = int(a[-1]) - 1
    row_2 = int(b[-1]) - 1

    selected_criteria = []
    for i in range(len(weighted_matrix.columns)):
        if weighted_matrix.iloc[row_1, i] < weighted_matrix.iloc[row_2, i]:
            selected_criteria.append([a, b, f"C{i+1}"])

    is_a_found = any(row[0] == a for row in selected_criteria)
    is_b_found = any(row[1] == b for row in selected_criteria)

    if is_a_found and is_b_found:
        for i in selected_criteria:
            discordances_data.append(i)
    else:
        discordances_data.append([a, b, None])

discordances = pd.DataFrame(discordances_data, columns=['First', 'Second', 'Selected Criteria'])
print(discordances)

   First Second Selected Criteria
0     A1     A2                C1
1     A1     A2                C2
2     A1     A2                C4
3     A1     A3                C1
4     A1     A3                C2
5     A1     A3                C3
6     A1     A4                C2
7     A1     A4                C3
8     A1     A4                C5
9     A1     A5                C2
10    A1     A5                C5
11    A2     A1                C3
12    A2     A1                C5
13    A2     A3                C3
14    A2     A3                C5
15    A2     A4                C2
16    A2     A4                C3
17    A2     A4                C5
18    A2     A5                C2
19    A2     A5                C3
20    A2     A5                C5
21    A3     A1                C4
22    A3     A2                C1
23    A3     A2                C4
24    A3     A4                C2
25    A3     A4                C5
26    A3     A5                C2
27    A3     A5                C4
28    A3     A

# Determine the Dominant Aggregate of The Matrix

In [ ]:
print('Calculation should be here')

# Eliminate Less Favourable Alternatives

In [ ]:
print('Calculation should be here')

# Rangking Alternative

In [ ]:
print('Calculation should be here')

---

# Conclusion